# Sacar imgs de carpetas

En este cuaderno es donde hemos creado el dataset final para poder hacer el *fine-tunning* y poder sacar las imágenes para **Roboflow**, tambien el poder sacar las imagenes del dataset de las imagenes random.

In [ ]:
import os
import shutil
import random

In [ ]:
# Función para copiar las primeras n imágenes de un directorio a otro
def save_first_n_images_to_folder(source_folder, destination_folder, n=10000):
    # Crear el directorio de destino si no existe
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Extensiones de las imagenes
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')

    # List to store all image paths
    image_paths = []

    # Recorrer el directorio de origen y guardar las rutas de las imagenes
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_paths.append(os.path.join(root, file))

    # Mezclar las imgs
    random.shuffle(image_paths)

    # Selecione las primeras n imagenes
    selected_images = image_paths[:n]

    # Copias las imagenes seleccionadas al directorio de destino
    for idx, source_path in enumerate(selected_images):
        file_extension = os.path.splitext(source_path)[1]
        destination_path = os.path.join(destination_folder, f"image_{idx}{file_extension}")
        shutil.copy2(source_path, destination_path)
        print(f"Copied {source_path} to {destination_path}")

# Uso de la función
source_folder = 'original'
destination_folder = 'primeras'
save_first_n_images_to_folder(source_folder, destination_folder, n=10000)

Copied original\train\MANGROVE SKIPPER\014.jpg to primeras\image_0.jpg
Copied original\train\LUNA MOTH\035.jpg to primeras\image_1.jpg
Copied original\train\CRECENT\115.jpg to primeras\image_2.jpg
Copied original\train\MONARCH\008.jpg to primeras\image_3.jpg
Copied original\train\JULIA\022.jpg to primeras\image_4.jpg
Copied original\train\YELLOW SWALLOW TAIL\005.jpg to primeras\image_5.jpg
Copied original\train\COMMON BANDED AWL\010.jpg to primeras\image_6.jpg
Copied original\train\CABBAGE WHITE\018.jpg to primeras\image_7.jpg
Copied original\train\EMPEROR GUM MOTH\070.jpg to primeras\image_8.jpg
Copied original\train\SCARCE SWALLOW\121.jpg to primeras\image_9.jpg
Copied original\train\APPOLLO\006.jpg to primeras\image_10.jpg
Copied original\train\ORANGE OAKLEAF\077.jpg to primeras\image_11.jpg
Copied original\train\AFRICAN GIANT SWALLOWTAIL\090.jpg to primeras\image_12.jpg
Copied original\train\ATALA\033.jpg to primeras\image_13.jpg
Copied original\train\RED SPOTTED PURPLE\014.jpg to 

# Contar la cantidad de imagenes en una carpeta

Estas seran las imagenes que le pasaremos a **Roboflow** para poder pasar las imágenes a formato YOLO y hacer el *fine-tunning*.

In [ ]:
def count_images_in_folder(folder):
    # Extensiones de las imagenes
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')

    # Contador de imágenes
    image_count = 0

    # Recorrer el directorio y contar las imagenes
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_count += 1

    return image_count

# Uso de la funcion
folder = 'primeras'
image_count = count_images_in_folder(folder)
print(f"Total number of images in '{folder}': {image_count}")

Total number of images in 'primeras': 10000


# Editar las imagenes random para ponerlas 640 x 640 y coger solo 200

Esta parte del cuaderno es para las imagenes random ya que no necesitan que esten en formato YOLO, solo con estar en el formato de tamaño que necesita el modelo para entrenar nos sirve, sacamos 200 ya que en total con las que nos da **Roboflow** serían unas 1200 imagenes.

In [ ]:
from PIL import Image

In [ ]:
def load_images_from_folder(folder, num_images):
    # Lista para almacenar las imágenes
    images = []
    # Recorrer el directorio y cargar las imágenes
    for filename in os.listdir(folder):
        if len(images) >= num_images:
            break
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = Image.open(img_path)
            images.append(img)
    return images

def resize_image(image, size):
    # Redimensionar la imagen
    return image.resize(size, Image.LANCZOS)

def save_image(image, path):
    # Guardar la imagen
    image.save(path)

def process_images(input_folder, output_folder, num_images=200, size=(640, 640)):
    # Crear el directorio de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Cargar las imágenes del directorio de entrada
    images = load_images_from_folder(input_folder, num_images)

    # Procesar las imágenes
    for i, image in enumerate(images):
        resized_image = resize_image(image, size)
        output_path = os.path.join(output_folder, f"resized_image_{i+1}.jpg")
        save_image(resized_image, output_path)

# Uso de la función
input_folder = "random_img/dataset/train"
output_folder = "random_img/editados"
process_images(input_folder, output_folder, num_images=200, size=(640, 640))

# Comprobamos que las imagenes esten bien y no sean nulas

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
def verificar_imagenes(directorio):
    """
    Verifica que las imágenes en un directorio sean válidas.
    
    Parámetros:
        directorio (str): Ruta del directorio donde están las imágenes.
        
    Retorna:
        imágenes_corruptas (list): Lista con las rutas de las imágenes corruptas o no válidas.
    """
    imagenes_corruptas = []
    formatos_validos = {".jpg", ".jpeg", ".png", ".bmp", ".tiff"}

    for archivo in tqdm(os.listdir(directorio), desc="Verificando imágenes"):
        ruta_imagen = os.path.join(directorio, archivo)

        # Verificar que sea un archivo de imagen válido por extensión
        if not any(archivo.lower().endswith(ext) for ext in formatos_validos):
            print(f"Archivo no válido (extensión incorrecta): {ruta_imagen}")
            imagenes_corruptas.append(ruta_imagen)
            continue

        # Intentar abrir la imagen con OpenCV
        try:
            imagen = cv2.imread(ruta_imagen)
            if imagen is None or imagen.size == 0:
                print(f"Imagen corrupta o vacía: {ruta_imagen}")
                imagenes_corruptas.append(ruta_imagen)
        except Exception as e:
            print(f"Error al leer la imagen {ruta_imagen}: {e}")
            imagenes_corruptas.append(ruta_imagen)

    print(f"Total de imágenes corruptas/no válidas encontradas: {len(imagenes_corruptas)}")
    return imagenes_corruptas

# Uso del código
directorio_imagenes = "ruta/a/tu/dataset"
imagenes_invalidas = verificar_imagenes(directorio_imagenes)

# para eliminar las imagenes corruptas
if imagenes_invalidas:
    for img in imagenes_invalidas:
        os.remove(img)
        print(f"Eliminada imagen corrupta: {img}")

print("Verificación completada.")
